In [1]:
from typing import List
import pyspark
pyspark.__version__

'3.1.1'

In [2]:
spark = pyspark.sql.SparkSession.builder \
                    .master("local[6]") \
                    .appName("Calcul des variations historiques de température par commune") \
                    .config("spark.driver.memory", "5g") \
                    .config("spark.executor.memory", "5g") \
                    .getOrCreate()

In [3]:
spark


In [4]:
type(spark)

pyspark.sql.session.SparkSession

In [5]:
from pyspark.sql.functions import col

In [6]:
dataframe = spark.range(1 * 10000000).toDF("id").withColumn("square", col("id") ** 2)
dataframe.persist()

DataFrame[id: bigint, square: double]

In [7]:
%time dataframe.count()

CPU times: user 773 µs, sys: 1.12 ms, total: 1.89 ms
Wall time: 2.5 s


10000000

In [8]:
%time dataframe.count()


CPU times: user 1.02 ms, sys: 1.59 ms, total: 2.61 ms
Wall time: 147 ms


10000000

In [ ]:
dataframe.persist(storageLevel="MEMORY_ONLY")


In [112]:
print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.appName)
print(spark.sparkContext.master)
print(spark.sparkContext.version)

http://amac02dq35hmd6r:4040
Calcul des variations historiques de température par commune
local[6]
3.0.2


## Meteo Dataset

In [77]:
meteo_file_path = "../dataset/extract_meteo.csv"
meteo_dataframe = spark.read.csv(path=meteo_file_path,
                                 header=True,
                                 sep=";")
meteo_dataframe.printSchema()


root
 |-- ID OMM station: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Pression au niveau mer: string (nullable = true)
 |-- Variation de pression en 3 heures: string (nullable = true)
 |-- Type de tendance barométrique4: string (nullable = true)
 |-- Direction du vent moyen 10 mn: string (nullable = true)
 |-- Vitesse du vent moyen 10 mn: string (nullable = true)
 |-- Température: string (nullable = true)
 |-- Point de rosée: string (nullable = true)
 |-- Humidité: string (nullable = true)
 |-- Visibilité horizontale: string (nullable = true)
 |-- Temps présent11: string (nullable = true)
 |-- Temps passé 112: string (nullable = true)
 |-- Temps passé 2: string (nullable = true)
 |-- Nebulosité totale: string (nullable = true)
 |-- Nébulosité  des nuages de l' étage inférieur: string (nullable = true)
 |-- Hauteur de la base des nuages de l'étage inférieur: string (nullable = true)
 |-- Type des nuages de l'étage inférieur: string (nullable = true)
 |-- Type des n

In [78]:
meteo_dataframe.count()

99999

In [79]:
from pyspark.sql.functions import col, year, month

meteo_dataframe_subset = meteo_dataframe.select(col("Date").cast("DATE"),
                                                col("mois_de_l_annee").cast("INT"),
                                                col("communes (name)").cast("STRING").alias("Communes"),
                                                col("communes (code)").cast("STRING").alias("CodePostal"),
                                                col("Température").cast("INT").alias("Temperature"))

meteo_dataframe_subset = meteo_dataframe_subset.select(year("Date").alias("year"),
                                                       month("Date").alias("month"),
                                                       "Date",
                                                       "Communes",
                                                       "CodePostal",
                                                       "Temperature")

print(meteo_dataframe_subset.printSchema())
meteo_dataframe_subset.show(5, truncate=False)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Communes: string (nullable = true)
 |-- CodePostal: string (nullable = true)
 |-- Temperature: integer (nullable = true)

None
+----+-----+----------+-----------------------+----------+-----------+
|year|month|Date      |Communes               |CodePostal|Temperature|
+----+-----+----------+-----------------------+----------+-----------+
|2013|4    |2013-04-03|Matoury                |97307     |302        |
|2013|4    |2013-04-03|Carpiquet              |14137     |277        |
|2013|4    |2013-04-03|Thuilley-aux-Groseilles|54523     |278        |
|2013|4    |2013-04-03|Perpignan              |66136     |286        |
|2013|4    |2013-04-03|Bangor                 |56009     |276        |
+----+-----+----------+-----------------------+----------+-----------+
only showing top 5 rows



## Filtre sur les communes non renseignées

In [80]:
print(meteo_dataframe_subset.count())
meteo_dataframe_subset = meteo_dataframe_subset.where(col("Communes").isNotNull())

99999


In [81]:
meteo_dataframe_subset.count()

86901

## INSEE Dataset

In [82]:
insee_file_path = "../dataset/MDB-INSEE-V2.csv"
insee_dataframe = spark.read.csv(path=insee_file_path,
                                 header=True,
                                 sep=";")
insee_dataframe.printSchema()

root
 |-- CODGEO: string (nullable = true)
 |-- Nb Pharmacies et parfumerie: string (nullable = true)
 |-- Dynamique Entrepreneuriale: string (nullable = true)
 |-- Dynamique Entrepreneuriale Service et Commerce: string (nullable = true)
 |-- Synergie Médicale COMMUNE: string (nullable = true)
 |-- Orientation Economique: string (nullable = true)
 |-- Indice Fiscal Partiel: string (nullable = true)
 |-- Score Fiscal: string (nullable = true)
 |-- Indice Evasion Client: string (nullable = true)
 |-- Score Evasion Client: string (nullable = true)
 |-- Indice Synergie Médicale: string (nullable = true)
 |-- Score Synergie Médicale: string (nullable = true)
 |-- SEG Croissance POP: string (nullable = true)
 |-- LIBGEO: string (nullable = true)
 |-- REG: string (nullable = true)
 |-- DEP: string (nullable = true)
 |-- Nb Omnipraticiens BV: string (nullable = true)
 |-- Nb Infirmiers Libéraux BV: string (nullable = true)
 |-- Nb dentistes Libéraux BV: string (nullable = true)
 |-- Nb pharmac

In [83]:
insee_dataframe_subset = insee_dataframe.select(col("CODGEO").cast("INT").alias("CodePostal"),
                                                #col("LIBGEO").cast("STRING").alias("Communes"),
                                                col("Population").cast("INT"))
print(insee_dataframe_subset.printSchema())
insee_dataframe_subset.show(5, truncate=False)

root
 |-- CodePostal: integer (nullable = true)
 |-- Population: integer (nullable = true)

None
+----------+----------+
|CodePostal|Population|
+----------+----------+
|1001      |725       |
|1002      |167       |
|1004      |11432     |
|1005      |1407      |
|1006      |86        |
+----------+----------+
only showing top 5 rows



## Filtre les communes de plus de 100 000 habitants

* Jointure des deux tables sur le code postal

In [84]:
insee_and_meteo_dataframe = meteo_dataframe_subset.join(insee_dataframe_subset,
                                                        on="CodePostal",
                                                        how='left')

insee_and_meteo_dataframe.show()

+----------+----+-----+----------+--------------------+-----------+----------+
|CodePostal|year|month|      Date|            Communes|Temperature|Population|
+----------+----+-----+----------+--------------------+-----------+----------+
|     97307|2013|    4|2013-04-03|             Matoury|        302|     18037|
|     14137|2013|    4|2013-04-03|           Carpiquet|        277|      1848|
|     54523|2013|    4|2013-04-03|Thuilley-aux-Gros...|        278|       219|
|     66136|2013|    4|2013-04-03|           Perpignan|        286|    105096|
|     56009|2013|    4|2013-04-03|              Bangor|        276|       742|
|     86194|2013|    4|2013-04-03|            Poitiers|        278|     83507|
|     87085|2013|    4|2013-04-03|             Limoges|        278|    133924|
|     65284|2013|    4|2013-04-03|               Louey|        282|       910|
|     97230|2013|    4|2013-04-03|          La Trinité|        300|     12883|
|     65284|2013|    4|2013-04-04|               Lou

In [85]:
insee_and_meteo_dataframe.printSchema()


root
 |-- CodePostal: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Communes: string (nullable = true)
 |-- Temperature: integer (nullable = true)
 |-- Population: integer (nullable = true)



In [86]:
insee_and_meteo_dataframe = insee_and_meteo_dataframe.where(col("Population") >= 1000)
insee_and_meteo_dataframe.show(5, truncate=False)

+----------+----+-----+----------+---------+-----------+----------+
|CodePostal|year|month|Date      |Communes |Temperature|Population|
+----------+----+-----+----------+---------+-----------+----------+
|97307     |2013|4    |2013-04-03|Matoury  |302        |18037     |
|14137     |2013|4    |2013-04-03|Carpiquet|277        |1848      |
|66136     |2013|4    |2013-04-03|Perpignan|286        |105096    |
|86194     |2013|4    |2013-04-03|Poitiers |278        |83507     |
|87085     |2013|4    |2013-04-03|Limoges  |278        |133924    |
+----------+----+-----+----------+---------+-----------+----------+
only showing top 5 rows



In [87]:
import pyspark.sql.functions as F

insee_and_meteo_dataframe.groupby("Communes", "year", "month").agg(F.max("Temperature")).show(5, truncate=False)


+----------------+----+-----+----------------+
|Communes        |year|month|max(Temperature)|
+----------------+----+-----+----------------+
|Perpignan       |2010|1    |285             |
|Mont-de-Marsan  |2010|2    |290             |
|Perpignan       |2010|2    |289             |
|Clermont-Ferrand|2011|4    |298             |
|Bourges         |2011|9    |300             |
+----------------+----+-----+----------------+
only showing top 5 rows



In [89]:
df_agg = insee_and_meteo_dataframe.groupby("Communes", "year", "month").agg(F.max("Temperature"))
df_agg.groupby("Communes", "month").agg(F.stddev_samp("max(Temperature)")).show(truncate=False)

+-------------------------+-----+-----------------------------+
|Communes                 |month|stddev_samp(max(Temperature))|
+-------------------------+-----+-----------------------------+
|Millau                   |1    |2.006932429798716            |
|Boos                     |10   |3.3990544903798576           |
|Sainte-Marie             |4    |1.2247448713915856           |
|Lorp-Sentaraille         |10   |2.0528725518856965           |
|Mauguio                  |2    |2.455153310442704            |
|Guipavas                 |1    |1.0540925533894523           |
|Perpignan                |12   |1.7268882005337967           |
|Holtzheim                |12   |2.5877458475338253           |
|Saint-Georges            |9    |0.8864052604279169           |
|Saint-Aignan-Grandlieu   |6    |3.240370349203916            |
|Boos                     |1    |2.635231383473653            |
|La Désirade              |10   |1.5275252316519468           |
|Montélimar               |9    |2.10017

In [88]:
insee_and_meteo_dataframe.groupby("Communes", "year", "month").agg(F.max("Temperature"))\
    .where((col("Communes") == "Clermont-Ferrand") & (col("month") == 1)).show()

+----------------+----+-----+----------------+
|        Communes|year|month|max(Temperature)|
+----------------+----+-----+----------------+
|Clermont-Ferrand|2012|    1|             283|
|Clermont-Ferrand|2017|    1|             281|
|Clermont-Ferrand|2015|    1|             287|
|Clermont-Ferrand|2016|    1|             282|
|Clermont-Ferrand|2013|    1|             281|
|Clermont-Ferrand|2010|    1|             281|
|Clermont-Ferrand|2018|    1|             285|
|Clermont-Ferrand|2014|    1|             285|
|Clermont-Ferrand|2011|    1|             287|
+----------------+----+-----+----------------+



In [52]:
insee_and_meteo_dataframe.printSchema()


root
 |-- Communes: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- max(Temperature): integer (nullable = true)



In [90]:
df_agg.explain()


== Physical Plan ==
*(3) HashAggregate(keys=[Communes#3600, year#3610, month#3611], functions=[max(Temperature#3602)])
+- Exchange hashpartitioning(Communes#3600, year#3610, month#3611, 200), true, [id=#2044]
   +- *(2) HashAggregate(keys=[Communes#3600, year#3610, month#3611], functions=[partial_max(Temperature#3602)])
      +- *(2) Project [year#3610, month#3611, Communes#3600, Temperature#3602]
         +- *(2) BroadcastHashJoin [cast(CodePostal#3601 as int)], [CodePostal#3888], Inner, BuildRight
            :- *(2) Project [year(cast(Date#3186 as date)) AS year#3610, month(cast(Date#3186 as date)) AS month#3611, communes (name)#3258 AS Communes#3600, communes (code)#3259 AS CodePostal#3601, cast(Température#3192 as int) AS Temperature#3602]
            :  +- *(2) Filter (isnotnull(communes (name)#3258) AND isnotnull(communes (code)#3259))
            :     +- FileScan csv [Date#3186,Température#3192,communes (name)#3258,communes (code)#3259] Batched: false, DataFilters: [isnotnull(

In [ ]:
import pyspark.sql.functions as F

(fire_ts_df.select(F.sum("NumAlarms"), F.avg("NumAlarms"), F.min("NumAlarms"), F.max("NumAlarms")).show())

avg_df = data_df.groupBy("name").agg(avg("age"))
.agg(countDistinct("CallType").alias("DisctinctCallType"))\
    .where(col("CallType").isNotNull())

In [ ]:
# In Python
fire_ts_df = (fire_df_pt
                .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate")
                .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate")
                .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
                .drop("AvailableDtTm"))


In [ ]:
%%time
# In Python to save as a Parquet file
parquet_path = "dataset/fire-calls.parquet"
fire_df.repartition(12)
fire_df.write.format("parquet").save(parquet_path)

In [ ]:
fire_df_pt = spark.read.parquet(parquet_path)
fire_df_pt.count()
fire_df_pt.printSchema()

fire_df.rdd.getNumPartitions()

In [93]:
%%time
from pyspark.sql.functions import col, year, month
import pyspark.sql.functions as F

#meteo_file_path = "../dataset/extract_meteo.csv"
meteo_file_path = "../dataset/donnees-synop-essentielles-omm.csv"
meteo_dataframe = spark.read.csv(path=meteo_file_path,
                                 header=True,
                                 sep=";")

meteo_dataframe_subset = meteo_dataframe.select(col("Date").cast("DATE"),
                                                col("mois_de_l_annee").cast("INT"),
                                                col("communes (name)").cast("STRING").alias("Communes"),
                                                col("communes (code)").cast("STRING").alias("CodePostal"),
                                                col("Température").cast("INT").alias("Temperature"))
meteo_dataframe_subset = meteo_dataframe_subset.select(year("Date").alias("year"),
                                                       month("Date").alias("month"),
                                                       "Date",
                                                       "Communes",
                                                       "CodePostal",
                                                       "Temperature")

meteo_dataframe_subset = meteo_dataframe_subset.where(col("Communes").isNotNull())

insee_file_path = "../dataset/MDB-INSEE-V2.csv"
insee_dataframe = spark.read.csv(path=insee_file_path,
                                 header=True,
                                 sep=";")

insee_dataframe_subset = insee_dataframe.select(col("CODGEO").cast("INT").alias("CodePostal"),
                                                #col("LIBGEO").cast("STRING").alias("Communes"),
                                                col("Population").cast("INT"))

insee_and_meteo_dataframe = meteo_dataframe_subset.join(insee_dataframe_subset,
                                                        on="CodePostal",
                                                        how='left')

insee_and_meteo_dataframe = insee_and_meteo_dataframe.where(col("Population") >= 1000)

df_agg = insee_and_meteo_dataframe.groupby("Communes", "year", "month").agg(F.max("Temperature"))
df_agg.groupby("Communes", "month").agg(F.stddev_samp("max(Temperature)")).collect()

CPU times: user 13.3 ms, sys: 3.86 ms, total: 17.2 ms
Wall time: 9.21 s


[Row(Communes='Millau', month=1, stddev_samp(max(Temperature))=2.1087839379532722),
 Row(Communes='Boos', month=10, stddev_samp(max(Temperature))=2.686667418602789),
 Row(Communes='Sainte-Marie', month=4, stddev_samp(max(Temperature))=0.40451991747794575),
 Row(Communes='Lorp-Sentaraille', month=10, stddev_samp(max(Temperature))=2.06265495285698),
 Row(Communes='Mauguio', month=2, stddev_samp(max(Temperature))=2.6097137890209536),
 Row(Communes='Guipavas', month=1, stddev_samp(max(Temperature))=1.3142574813455559),
 Row(Communes='Perpignan', month=12, stddev_samp(max(Temperature))=1.2720777563426902),
 Row(Communes='Holtzheim', month=12, stddev_samp(max(Temperature))=2.3001976199685736),
 Row(Communes='Saint-Georges', month=9, stddev_samp(max(Temperature))=0.6741998624632584),
 Row(Communes='Saint-Aignan-Grandlieu', month=6, stddev_samp(max(Temperature))=3.9450774851245085),
 Row(Communes='Boos', month=1, stddev_samp(max(Temperature))=1.5447859516333022),
 Row(Communes='La Désirade', m

In [115]:
df_agg.write.csv()


DataFrame[Communes: string, year: int, month: int, max(Temperature): int]

In [127]:
%%time
import random

n: int = 10000000
foo_list = [random.random() for _ in range(n)]
foo_list.sort()

a: float = 0.25
b: float = 0.27

CPU times: user 836 ms, sys: 44 ms, total: 880 ms
Wall time: 890 ms


In [150]:
def dichomomous_search2(foo_list: List[float], a: float, b: float, index_start: int, index_end: int) -> List[float]:
    n_element: int = index_end - index_start
    if n_element == 0:
        return []
    mid_elementindex: int = n_element // 2 + index_start
    mid_element = foo_list[mid_elementindex]
    if mid_element < a:
        return dichomomous_search2(foo_list, a, b, index_start, mid_elementindex)
    elif mid_element > b:
        return dichomomous_search2(foo_list, a, b, mid_elementindex + 1, index_end)
    else:
        return [elt for elt in foo_list if a <= elt and elt < b]

In [153]:
%%timeit
n= 10000000
foo_list = [random.random() for _ in range(n)]
foo_list.sort()
a = 0.25
b = 0.27
dichomomous_search2(foo_list, a, b, 0, len(foo_list))

4.68 s ± 77.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
%%timeit
n = 10000000
foo_list = [random.random() for _ in range(n)]
foo_list_arr = np.sort(foo_list)
a = 0.25
b = 0.27
indexes = np.where(np.logical_and(foo_list_arr>=a, foo_list_arr<=b))
foo_list_arr[indexes]

2.28 s ± 63.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [146]:
indexes = np.where(np.logical_and(foo_list_arr>=a, foo_list_arr<=b))
foo_list_arr[indexes]

array([0.25000012, 0.25000022, 0.25000027, ..., 0.26999964, 0.26999985,
       0.26999986])

In [148]:
%timeit search_with_numpy(foo_list_arr)

15.9 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [134]:
search_with_numpy(foo_list)

array([0.2606772 , 0.26998024, 0.25399611, ..., 0.25311427, 0.2661466 ,
       0.25982513])

In [139]:
foo_list_arr

array([7.89048132e-08, 2.01010326e-07, 2.44992134e-07, ...,
       9.99999806e-01, 9.99999900e-01, 9.99999959e-01])

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List

In [10]:
class Consumption:
    def __init__(self, meter_number: str, start_date: datetime, end_date: datetime, quantity: float):
        self.meter_number: str = meter_number
        self.start_date: datetime = start_date
        self.end_date: datetime = end_date
        self.quantity: float = quantity

start_date: datetime = datetime(2015, 1, 1)
list_consumption: List[Consumption] = []
for i in range(300000):
    consumption_1 = Consumption(meter_number='LI1',
                              start_date = start_date + timedelta(minutes=10 * i),
                              end_date = start_date + timedelta(minutes=10 * (i +1)),
                              quantity = 1)
    consumption_2 = Consumption(meter_number='LI2',
                              start_date = start_date + timedelta(minutes=10 * i),
                              end_date = start_date + timedelta(minutes=10 * (i +1)),
                              quantity = 1)
    list_consumption.extend([consumption_1, consumption_2])

In [13]:
list_consumption[0].quantity

1

In [57]:
df = pd.DataFrame({'meter_number': [consumption.meter_number for consumption in list_consumption],
                   'start_date': [consumption.start_date for consumption in list_consumption],
                   'end_date': [consumption.end_date for consumption in list_consumption],
                   'quantity': [consumption.quantity for consumption in list_consumption]})

In [37]:
df.head(5)

,meter_number,start_date,end_date,quantity
0,LI1,2015-01-01 00:00:00,2015-01-01 00:10:00,1
1,LI2,2015-01-01 00:00:00,2015-01-01 00:10:00,1
2,LI1,2015-01-01 00:10:00,2015-01-01 00:20:00,1
3,LI2,2015-01-01 00:10:00,2015-01-01 00:20:00,1
4,LI1,2015-01-01 00:20:00,2015-01-01 00:30:00,1


In [31]:
df.dtypes

meter_number            object
start_date      datetime64[ns]
end_date        datetime64[ns]
quantity                 int64
dtype: object

In [60]:
%%timeit
start_date=datetime(2020, 1, 1)
end_date = datetime(2020, 3, 1)
df.iloc[np.where(np.logical_and(df.start_date >= start_date, df.end_date < end_date))[0]]


3.91 ms ± 427 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%time
start_date=datetime(2020, 1, 1)
end_date = datetime(2020, 3, 1)
df.set_index(['start_date', 'end_date'], inplace=True)
idx = pd.IndexSlice
df.loc[idx[start_date:, :end_date], :]

CPU times: user 102 ms, sys: 28.5 ms, total: 131 ms
Wall time: 130 ms


meter_number  quantity
start_date          end_date                                  
2020-01-01 00:00:00 2020-01-01 00:10:00          LI1         1
                    2020-01-01 00:10:00          LI2         1
2020-01-01 00:10:00 2020-01-01 00:20:00          LI1         1
                    2020-01-01 00:20:00          LI2         1
2020-01-01 00:20:00 2020-01-01 00:30:00          LI1         1
...                                              ...       ...
2020-02-29 23:30:00 2020-02-29 23:40:00          LI2         1
2020-02-29 23:40:00 2020-02-29 23:50:00          LI1         1
                    2020-02-29 23:50:00          LI2         1
2020-02-29 23:50:00 2020-03-01 00:00:00          LI1         1
                    2020-03-01 00:00:00          LI2         1

[17280 rows x 2 columns]

In [33]:
df


meter_number  quantity
start_date          end_date                                  
2015-01-01 00:00:00 2015-01-01 00:10:00          LI1         1
                    2015-01-01 00:10:00          LI2         1
2015-01-01 00:10:00 2015-01-01 00:20:00          LI1         1
                    2015-01-01 00:20:00          LI2         1
2015-01-01 00:20:00 2015-01-01 00:30:00          LI1         1
...                                              ...       ...
2020-09-14 07:30:00 2020-09-14 07:40:00          LI2         1
2020-09-14 07:40:00 2020-09-14 07:50:00          LI1         1
                    2020-09-14 07:50:00          LI2         1
2020-09-14 07:50:00 2020-09-14 08:00:00          LI1         1
                    2020-09-14 08:00:00          LI2         1

[600000 rows x 2 columns]

In [61]:
df = pd.DataFrame({'meter_number': [consumption.meter_number for consumption in list_consumption],
                   'start_date': [consumption.start_date for consumption in list_consumption],
                   'end_date': [consumption.end_date for consumption in list_consumption],
                   'quantity': [consumption.quantity for consumption in list_consumption]})
df.set_index(['start_date', 'end_date'], inplace=True)

In [62]:
%%timeit
start_date=datetime(2020, 1, 1)
end_date = datetime(2020, 3, 1)
idx = pd.IndexSlice
df.loc[idx[start_date:, :end_date], :]

11.3 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%%timeit
df.loc[idx[start_date:, :end_date], :]

11.8 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
